In [1]:
from langchain_upstage import UpstageEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [2]:
def kiwi_tokenize(text):
    from kiwipiepy import Kiwi
    kiwi = Kiwi()
    return [token.form for token in kiwi.tokenize(text)]

In [3]:
import sys
from pathlib import Path

# Add project root to path to import config
# Try multiple paths to find the project root (where config.py is located)
current_dir = Path().resolve()
possible_roots = [
    current_dir.parent,  # If running from reranker/ directory
    current_dir,         # If running from project root
]

for root in possible_roots:
    if (root / "config.py").exists():
        if str(root) not in sys.path:
            sys.path.insert(0, str(root))
        break

from config import output_path_prefix
import pickle


In [4]:
with open(f"{output_path_prefix}_split_documents.pkl", "rb") as f:
    split_documents = pickle.load(f)

In [5]:
len(split_documents)

61

In [6]:
embeddings = UpstageEmbeddings(model="embedding-passage")

In [7]:
bm25_retriever = BM25Retriever.from_documents(split_documents, preprocess_func=kiwi_tokenize)

In [8]:
bm25_retriever.k = 10

In [9]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [10]:
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [11]:
bm25_docs = bm25_retriever.invoke("본문에서 제시된 2022년 개정 교육과정 관련 3가지 핵심 사실은 무엇인가요?")

In [12]:
faiss_docs = faiss_retriever.invoke("본문에서 제시된 2022년 개정 교육과정 관련 3가지 핵심 사실은 무엇인가요?")

In [13]:
len(faiss_docs), len(bm25_docs)

(10, 10)

In [14]:
faiss_docs[:5]

[Document(id='a4ee20a2-48b0-4a56-85ab-416c2fba39f4', metadata={'page': 2, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 2}, page_content='JOURNAL OF MULTICULTURE AND EDUCATION (Vol.9, No.1)\n\nJOURNAL OF MULTICULTURE AND EDUCATION (Vol.9, No.1)\n\nㅎ\n\n  \n\nⅠ. 서론\n=====\n\n현대 교육 환경은 세계화와 정보화를 맞이하여 끊임없이 변화하고 있으며, 이는 사회과교  \n육의 영역에도 새로운 도전과 기회를 제시한다. 본 연구는 변화하는 교육의 패러다임에 직면  \n한 예비 사회과 교사들의 인식은 어떠한지 살펴보고자 한다. 2015년 개정 교육과정에서 강조  \n된 핵심 가치 중 하나는 인간상이다. 그 가운데 ‘자주적인 사람’이라는 표현은 2022년 개정  \n교육과정에서 ‘자기 주도적 사람’으로 변경되었다. 이는 2022년 개정 교육과정에서 ‘자기 주  \n도성’과 ‘역량 함양 교육과정’을 강조하고 있음을 나타낸다. 이러한 교육 원칙은 OECD의  \nEducation 2030과 밀접한 관련이 있으며, 그 맥락을 함께 고려해야 한다(임은진, 2023).  \n이런 맥락에서 사회과 역량은 비판적 사고력, 창의적 사고력, 의사 결정력 및 문제 해결력,  \n정보 활용 능력, 의사소통 및 협업 능력으로 설정되었다(교육부, 2022). 사회과 역량은 중등  \n교사 양성기관인 사범대학 사회교육과의 예비교사 역량을 위한 교육과정에 반영되어 교수학  \n습 활동에 실현된다. 특히 활동중심 교육과 문제해결중심 학습은 현지조사라는 형태로 실천된  \n다. 예비 사회과 교사들의 현지조사는 교육부에서 제시한 사회과 역량에 긍정적인 영향을  \n미친다(문희진, 2022). 본 연구는 사회과에서 강조하는 체험학습에

In [ ]:
k = 60

for idx, (faiss, bm25) in enumerate(zip(faiss_docs, bm25_docs)):
    # print(faiss.metadata['id'], bm25.metadata['id'])
    rank = idx + 1
    faiss.metadata['rank_score'] = 1 / (k + rank)
    bm25.metadata['rank_score'] = 1 / (k + rank)    
    

2 2
48 7
49 42
3 3
4 49
15 12
12 13
22 48
13 11
29 34


In [ ]:
for doc in bm25_docs:
    print(doc.metadata['rank_score'])

0.01639344262295082
0.016129032258064516
0.015873015873015872
0.015625
0.015384615384615385
0.015151515151515152
0.014925373134328358
0.014705882352941176
0.014492753623188406
0.014285714285714285


In [25]:
docs =faiss_docs + bm25_docs

In [26]:
len(docs)

20

In [27]:
merged = {}
for doc in docs:
    current_id = doc.metadata['id']

    new_id = True if current_id not in merged.keys() else False
    bucket = merged.setdefault(current_id, doc.model_copy(deep=True))
    
    if not new_id:
        bucket.metadata['rank_score'] += doc.metadata['rank_score']

    else:
        bucket.metadata['rank_score'] = doc.metadata['rank_score']

merged_docs = list(merged.values())



In [28]:
merged_docs

[Document(id='a4ee20a2-48b0-4a56-85ab-416c2fba39f4', metadata={'page': 2, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 2, 'rank_score': 0.03278688524590164}, page_content='JOURNAL OF MULTICULTURE AND EDUCATION (Vol.9, No.1)\n\nJOURNAL OF MULTICULTURE AND EDUCATION (Vol.9, No.1)\n\nㅎ\n\n  \n\nⅠ. 서론\n=====\n\n현대 교육 환경은 세계화와 정보화를 맞이하여 끊임없이 변화하고 있으며, 이는 사회과교  \n육의 영역에도 새로운 도전과 기회를 제시한다. 본 연구는 변화하는 교육의 패러다임에 직면  \n한 예비 사회과 교사들의 인식은 어떠한지 살펴보고자 한다. 2015년 개정 교육과정에서 강조  \n된 핵심 가치 중 하나는 인간상이다. 그 가운데 ‘자주적인 사람’이라는 표현은 2022년 개정  \n교육과정에서 ‘자기 주도적 사람’으로 변경되었다. 이는 2022년 개정 교육과정에서 ‘자기 주  \n도성’과 ‘역량 함양 교육과정’을 강조하고 있음을 나타낸다. 이러한 교육 원칙은 OECD의  \nEducation 2030과 밀접한 관련이 있으며, 그 맥락을 함께 고려해야 한다(임은진, 2023).  \n이런 맥락에서 사회과 역량은 비판적 사고력, 창의적 사고력, 의사 결정력 및 문제 해결력,  \n정보 활용 능력, 의사소통 및 협업 능력으로 설정되었다(교육부, 2022). 사회과 역량은 중등  \n교사 양성기관인 사범대학 사회교육과의 예비교사 역량을 위한 교육과정에 반영되어 교수학  \n습 활동에 실현된다. 특히 활동중심 교육과 문제해결중심 학습은 현지조사라는 형태로 실천된  \n다. 예비 사회과 교사들의 현지조사는 교육부에서 제시한 사회과 역량에 긍정적인 영향을  \n미친다

In [29]:
merged_docs_sorted = sorted(merged_docs, key=lambda x: x.metadata['rank_score'], reverse=True)
len(merged_docs_sorted)

14

In [30]:
for doc in merged_docs_sorted:
    print(doc.metadata['rank_score'])

0.03278688524590164
0.03125763125763126
0.03125
0.030834914611005692
0.03007688828584351
0.029418126757516764
0.016129032258064516
0.015873015873015872
0.015384615384615385
0.015151515151515152
0.014705882352941176
0.014492753623188406
0.014285714285714285
0.014285714285714285


In [32]:
merged_docs_sorted[1].page_content

'육학회지(구 지리환경교육), 22(1), 111-130.  \n원동윤, 정춘헌, 강근모(2021). 학교밖청소년지원센터 종사자의 소진극복방안에 관한 연구: Q  \n방법론적 접근. 한국청소년연구, 32(2), 237-265.  \n임은진(2023). 2022 개정 사회과 지리영역 교육과정 개발 방향 및 주요 내용. 한국지리환경교  \n육학회지, 31(1), 1-13.  \n정찬미, 신동희(2020). 야외 지질 답사 교육에서 나타난 귀추적 탐구 수행 특성 변화. J. Korean  \nEarth Sci. Soc, 41(5), 531-554.  \n최수안(2022). 좋은 자조모임에 대한 결혼이주여성의 인식 유형 탐색: Q 방법론의 적용. 여성  \n연구, 115, 335-362.  \n한동균(2020). 초등교사의 지속가능발전교육에 대한 인식 유형: Q방법론적 접근. 사회과교육,  \n59(4), 161-178.  \n한상복, 이문웅, 김광억(2011). 문화인류학. 서울: 서울대학교출판문화원.  \nBrown, R. (1980). Political subjectivity. New Haven, CT: Yale University Press. Retrieved'